In [ ]:
import json
import pathlib
import random
import functools

import numpy as np
import matplotlib.pyplot as plt

import skimage.filters

from IPython import display

In [ ]:
import tensorflow as tf

In [ ]:
# Makes it so any changes in pymedphys is automatically
# propagated into the notebook without needing a kernel reset.
from IPython.lib.deepreload import reload
%load_ext autoreload
%autoreload 2

In [ ]:
!pip install "pymedphys>=0.33.0"
!pip install "pydicom>=2"

In [ ]:
import pydicom

In [ ]:
import pymedphys
from pymedphys._experimental.autosegmentation import indexing, filtering, pipeline, mask, softdice

In [ ]:
ct_uid_to_use = '1.2.840.113704.1.111.3096.1537312918.112198'

In [ ]:
# Create masks for the following structures, in the following order
structures_to_learn = [
    'lens_left', 'lens_right', 'eye_left', 'eye_right', 'patient']

# Use the following to filter the slices used for training, validation,
# and testing
filters = {
    "study_set_must_have_all_of": structures_to_learn,
    "slice_at_least_one_of": [
        'lens_left', 'lens_right', 'eye_left', 'eye_right'
    ],
    "slice_must_have": ['patient'],
    "slice_cannot_have": []
}

In [ ]:
(
    data_path_root,
    structure_set_paths,
    ct_image_paths,
    ct_uid_to_structure_uid,
    _,
    names_map,
    _,
    _,
    uid_to_url,
    hash_path,
) = pipeline.get_dataset_metadata()

In [ ]:
@functools.lru_cache()
def get_dcm_ct_from_uid(ct_uid):
    ct_path = ct_image_paths[ct_uid]
    dcm_ct = pydicom.read_file(ct_path, force=True)

    dcm_ct.file_meta.TransferSyntaxUID = pydicom.uid.ImplicitVRLittleEndian

    return dcm_ct

In [ ]:
binary_mask_dataset = pipeline.create_dataset(
    [ct_uid_to_use], structures_to_learn, expansion=1)

floating_point_edge_dataset = pipeline.create_dataset(
    [ct_uid_to_use], structures_to_learn, expansion=5)

In [ ]:
def diagnostic_plotting(x_grid, y_grid, input_array, output_array):
    plt.figure(figsize=(15,10))
    
    x_grid = x_grid.numpy()
    y_grid = y_grid.numpy()
    input_array = input_array.numpy()[:,:,0]
    output_array = output_array.numpy()
    
    for i, structure in enumerate(structures_to_learn[0:-1]):
        if structure.endswith('left'):
            colour = 'r'
        elif structure.endswith('right'):
            colour = 'b'
        else:
            raise ValueError("Expected either left or right")
            
        if structure.startswith('lens'):
            colour += '--'
        elif structure.startswith('eye'):
            colour += '-'
        else:
            raise ValueError("Expected either eye or lens")

        contours = mask.get_contours_from_mask(
            x_grid, y_grid, output_array[:,:,i])
        for contour in contours:
            plt.plot(*contour.T, colour)
                        
    
    plt.axis('equal')
    ax = plt.gca()
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    
    contours = mask.get_contours_from_mask(
        x_grid, y_grid, output_array[:,:,-1])
    for contour in contours:
        plt.plot(*contour.T, 'k--')
    
    windowed = np.copy(input_array)

    vmin = 900
    vmax = 1200
    windowed[windowed<vmin] = vmin
    windowed[windowed>vmax] = vmax

    plt.contourf(x_grid, y_grid, windowed, 50)
    plt.colorbar()
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)

## Contours produced from binary masks

In [ ]:
for ct_uid, x_grid, y_grid, input_array, output_array in binary_mask_dataset:    
    ct_uid = ct_uid.numpy().decode()
    
    binary_mask_lens = output_array[:,:,0]
    
    display.display(display.Markdown(f"## {ct_uid}"))
    diagnostic_plotting(x_grid, y_grid, input_array, output_array)
    plt.show()

## Contours produced where the edge pixel of a mask is able to be a floating point

In [ ]:
for ct_uid, x_grid, y_grid, input_array, output_array in floating_point_edge_dataset:    
    ct_uid = ct_uid.numpy().decode()
    
    floating_point_mask_lens = output_array[:,:,0]
    
    display.display(display.Markdown(f"## {ct_uid}"))
    diagnostic_plotting(x_grid, y_grid, input_array, output_array)
    plt.show()

In [ ]:
contours = mask.get_contours_from_mask(
    x_grid, y_grid, binary_mask_lens[:,:])

for contour in contours:
    plt.plot(*contour.T)

plt.axis('equal')

In [ ]:
contours = mask.get_contours_from_mask(
    x_grid, y_grid, floating_point_mask_lens[:,:])

for contour in contours:
    plt.plot(*contour.T)

plt.axis('equal')

In [ ]:
new_contour = contour.copy()
new_contour[5,0] -= 0.1
new_contour[6,0] -= 0.1
new_contour[7,0] -= 0.1

In [ ]:
plt.plot(*contour.T)
plt.plot(*new_contour.T)

lims = plt.axis('equal')

In [ ]:
dcm_ct = get_dcm_ct_from_uid(ct_uid)

In [ ]:
def mock_dicom_contour(input_contour):
    contour_numpy = np.zeros((np.shape(input_contour)[0], 3))
    contour_numpy[:,:-1] = input_contour
    dicom_contour = np.ravel(contour_numpy)

    return [dicom_contour]

In [ ]:
binary_mask_original = mask.calculate_anti_aliased_mask(mock_dicom_contour(contour), dcm_ct, expansion=1)
binary_mask_edited = mask.calculate_anti_aliased_mask(mock_dicom_contour(new_contour), dcm_ct, expansion=1)

In [ ]:
x_grid, y_grid = binary_mask_original[0:2]

In [ ]:
plt.pcolormesh(*binary_mask_original)
plt.axis('equal')
plt.xlim(lims[0:2])
plt.ylim(lims[2:])

In [ ]:
plt.pcolormesh(*binary_mask_edited)
plt.axis('equal')
plt.xlim(lims[0:2])
plt.ylim(lims[2:])

In [ ]:
diff = binary_mask_original[-1] - binary_mask_edited[-1]

In [ ]:
plt.pcolormesh(x_grid, y_grid, diff)
plt.axis('equal')
plt.xlim(lims[0:2])
plt.ylim(lims[2:])

In [ ]:
floating_mask_original = mask.calculate_anti_aliased_mask(mock_dicom_contour(contour), dcm_ct, expansion=5)
floating_mask_edited = mask.calculate_anti_aliased_mask(mock_dicom_contour(new_contour), dcm_ct, expansion=5)

In [ ]:
plt.pcolormesh(*floating_mask_original)
plt.axis('equal')
plt.xlim(lims[0:2])
plt.ylim(lims[2:])

In [ ]:
plt.pcolormesh(*floating_mask_edited)
plt.axis('equal')
plt.xlim(lims[0:2])
plt.ylim(lims[2:])

In [ ]:
floating_diff = floating_mask_original[-1] - floating_mask_edited[-1]

In [ ]:
plt.pcolormesh(x_grid, y_grid, floating_diff)
plt.axis('equal')
plt.xlim(lims[0:2])
plt.ylim(lims[2:])

In [ ]:
binary_returned_contours = mask.get_contours_from_mask(x_grid, y_grid, binary_mask_edited[-1])
returned_contours = mask.get_contours_from_mask(x_grid, y_grid, floating_mask_edited[-1])

In [ ]:
plt.plot(*contour.T, '--', alpha=0.8)
plt.plot(*new_contour.T)
plt.plot(*returned_contours[-1].T)
plt.plot(*binary_returned_contours[-1].T, '--', alpha=0.8)

plt.axis('equal')

In [ ]:
edge_reference = skimage.filters.scharr(floating_mask_original[-1])

plt.pcolormesh(x_grid, y_grid, edge_reference)
plt.axis('equal')
plt.xlim(lims[0:2])
plt.ylim(lims[2:])

In [ ]:
edge_evaluation = skimage.filters.scharr(floating_mask_edited[-1])

plt.pcolormesh(x_grid, y_grid, edge_evaluation)
plt.axis('equal')
plt.xlim(lims[0:2])
plt.ylim(lims[2:])

In [ ]:
cost_diff = edge_evaluation - edge_reference

plt.pcolormesh(x_grid, y_grid, cost_diff)
plt.axis('equal')
plt.xlim(lims[0:2])
plt.ylim(lims[2:])

In [ ]:
score = np.sum(np.abs(edge_evaluation - edge_reference)) / np.sum(
    edge_evaluation + edge_reference
)

softdice = 1 - score

softdice